In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import math
from itertools import combinations,product

This section will look to solve the Traveling Salesman Problem through the Quadratic Assignment formulation. 

In [2]:
# Function will convert our datfile of x,y coordinates into dataframe: vertices
def TSP_vertices(file_name):
    data  = open(file_name, 'r')
    lines = data.readlines()[1:]
    x = []
    y = []
    for line in lines:
        p = line.split()
        x.append(float(p[0]))
        y.append(float(p[1]))
    data.close()
    # Transform into dataframe
    vertices = pd.DataFrame({'x':x, 'y':y})
    
    return vertices

In [3]:
# Will convert Euclidean distance for our coordinates as cost components for model
def euclideanDistMatrix(TSP):
    euclidean_cost = []
    for i in range(len(TSP)):
        row_distances = []
        for j in range(len(TSP)):
            row_distances.append(math.sqrt( (TSP['x'][i]-TSP['x'][j])**2 + (TSP['y'][i]-TSP['y'][j])**2 ))
        euclidean_cost.append(row_distances)
    return euclidean_cost  

In [4]:
def quadratic_assignment(file_name):

    # Turn .dat file into a cost matrix with all costs associated from traversing from i to j
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list V that contains all the vertices, numbered 0 through n-1
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of (i,k) which represents assigning vertex i to box k in the permutation box
    # Create a list of tuples (i,j,k) for w[i,j,k] decision variable which will help linearize our objective function
    A = [(i,k) for i in V for k in V]
    W = [(i,j,k) for i in V for j in V for k in V if j != i]

    # Instantiate the Model
    model = Model('Quadratic Assignment')
    model.ModelSense = GRB.MINIMIZE

    # Instantiate the decision variables
    x = model.addVars(A, vtype = GRB.BINARY, name='e')
    w = model.addVars(W, vtype = GRB.INTEGER, name='w')

    # Set the objective function
    model.modelSense = GRB.MINIMIZE
    model.setObjective( quicksum(cost[i][j]*w[i,j,k] for k in V[0:len(V)-1] for j in V for i in V if i != j) 
                  + quicksum(cost[i][j]*w[i,j,len(V)-1] for j in V for i in V if i != j) )

    # Constraint 1: Value of w[i,j,k] based off x[i,k] and x[j,k+1] 
    model.addConstrs(w[i,j,k] >= x[i,k] + x[j,k+1] - 1 for i in V for j in V for k in V[0:len(V)-1] if i != j)
    # Constraint 2: Value of w[i,j,n] based off x[i,n] and x[j,0] where k=n represents the last box 
    model.addConstrs(w[i,j,len(V)-1] >= x[i,len(V)-1] + x[j,0] - 1 for j in V for i in V if i != j)
    # Constraint 3: leaving node i
    model.addConstrs(quicksum(x[i,j] for j in V) == 1 for i in V)
    # Constraint 4: entering node j
    model.addConstrs(quicksum(x[i,j] for i in V) == 1 for j in V)

    # Solve the model and print runtime
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')

    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('Quadratic Assignment.lp')
    
    ### UNCOMMENT BELOW if you want the positive decision variables printed out to the user 
    #for v in model.getVars():
    #   if v.x > 0:
    #  print(v)
            
    return model

In [19]:
def quadratic_assignment_LR(file_name):

    # Turn .dat file into a cost matrix with all costs associated from traversing from i to j
    verticesDF = TSP_vertices(file_name)
    cost = euclideanDistMatrix(verticesDF)

    # Create a list V that contains all the vertices, numbered 0 through n-1
    V = []
    for i in range(len(verticesDF)):
        V.append(i)
    
    # Create a list A of all tuples of (i,k) which represents assigning vertex i to box k in the permutation box
    # Create a list of tuples (i,j,k) for w[i,j,k] decision variable which will help linearize our objective function 
    A = [(i,k) for i in V for k in V]
    W = [(i,j,k) for i in V for j in V for k in V if j != i]

    # Instantiate the Model
    model = Model('Quadratic Assignment')
    model.ModelSense = GRB.MINIMIZE

    # Instantiate the decision variables
    x = model.addVars(A, vtype = GRB.BINARY, name='e')
    w = model.addVars(W, lb=0, ub=1, vtype = GRB.CONTINUOUS, name='w')

    # Set the objective function
    model.modelSense = GRB.MINIMIZE
    model.setObjective( quicksum(cost[i][j]*w[i,j,k] for k in V[0:len(V)-1] for j in V for i in V if i != j) 
                  + quicksum(cost[i][j]*w[i,j,len(V)-1] for j in V for i in V if i != j) )

    # Constraint 1: Value of w[i,j,k] based off x[i,k] and x[j,k+1]  
    model.addConstrs(w[i,j,k] >= x[i,k] + x[j,k+1] - 1 for i in V for j in V for k in V[0:len(V)-1] if i != j)
    # Constraint 2: Value of w[i,j,n] based off x[i,n] and x[j,0] where k=n represents the last box
    model.addConstrs(w[i,j,len(V)-1] >= x[i,len(V)-1] + x[j,0] - 1 for j in V for i in V if i != j)
    # Constraint 3: leaving node i
    model.addConstrs(quicksum(x[i,j] for j in V) == 1 for i in V)
    # Constraint 4: entering node j
    model.addConstrs(quicksum(x[i,j] for i in V) == 1 for j in V)

    # Solve the model and print runtime
    model.Params.TimeLimit = 600
    model.optimize()
    print('The runtime was', model.Runtime, 'seconds.')

    ### UNCOMMENT LINE BELOW if you want to save a file of the formulation. Open with Microsoft Word.
    # model.write('Quadratic Assignment.lp')
    
    ### UNCOMMENT BELOW if you want the positive decision variables printed out to the user 
    #for v in model.getVars():
    #   if v.x > 0:
    #  print(v)
            
    return model

The following are the Quadratic Assignment formulation integer and Linear Relaxation solutions for two instances of size 15.

In [6]:
quadratic_assignment('TSP_instance_n_15_s_1.dat')

Using license file /Users/student/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3375 columns and 9900 nonzeros
Model fingerprint: 0x36224a53
Variable types: 0 continuous, 3375 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 111.0339672
Presolve time: 0.03s
Presolved: 3180 rows, 3375 columns, 9900 nonzeros
Variable types: 0 continuous, 3375 integer (3375 binary)

Root relaxation: objective 0.000000e+00, 145 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   43  111.03397    0.0

<gurobi.Model MIP instance Quadratic Assignment: 3180 constrs, 3375 vars, Parameter changes: TimeLimit=600.0>

In [20]:
quadratic_assignment_LR('TSP_instance_n_15_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3375 columns and 9900 nonzeros
Model fingerprint: 0xa6f4eecf
Variable types: 3150 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 111.0339672
Presolve time: 0.03s
Presolved: 3180 rows, 3375 columns, 13050 nonzeros
Variable types: 3150 continuous, 225 integer (225 binary)

Root relaxation: objective 0.000000e+00, 175 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   30  111.03397    0.00000   100%     -    0s
     0     0    0.00000    0   41  111.03397    0.00000   100%  

<gurobi.Model MIP instance Quadratic Assignment: 3180 constrs, 3375 vars, Parameter changes: TimeLimit=600.0>

In [21]:
quadratic_assignment('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3375 columns and 9900 nonzeros
Model fingerprint: 0x5eaacf60
Variable types: 0 continuous, 3375 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 183.5511682
Presolve time: 0.03s
Presolved: 3180 rows, 3375 columns, 9900 nonzeros
Variable types: 0 continuous, 3375 integer (3375 binary)

Root relaxation: objective 0.000000e+00, 145 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   43  183.55117    0.00000   100%     -    0s
H    0     0                     157.7845366    0.00000   100%     -

<gurobi.Model MIP instance Quadratic Assignment: 3180 constrs, 3375 vars, Parameter changes: TimeLimit=600.0>

In [22]:
quadratic_assignment_LR('TSP_instance_n_15_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3180 rows, 3375 columns and 9900 nonzeros
Model fingerprint: 0x7d806753
Variable types: 3150 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 183.5511682
Presolve time: 0.04s
Presolved: 3180 rows, 3375 columns, 13050 nonzeros
Variable types: 3150 continuous, 225 integer (225 binary)

Root relaxation: objective 0.000000e+00, 153 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   47  183.55117    0.00000   100%     -    0s
H    0     0                     170.4964698    0.00000   100%  

<gurobi.Model MIP instance Quadratic Assignment: 3180 constrs, 3375 vars, Parameter changes: TimeLimit=600.0>

The following are the Quadratic Assignment formulation integer and Linear Relaxation solutions for two instances of size 20.

In [23]:
quadratic_assignment('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 8000 columns and 23600 nonzeros
Model fingerprint: 0xde2dd0f4
Variable types: 0 continuous, 8000 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 195.6513711
Presolve time: 0.06s
Presolved: 7640 rows, 8000 columns, 23600 nonzeros
Variable types: 0 continuous, 8000 integer (8000 binary)

Root relaxation: objective 0.000000e+00, 290 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   60  195.65137    0.00000   100%     -    0s
H    0     0                     177.1140693    0.00000   100%    

  7834  3995   64.76673   33  164   71.80154   58.89859  18.0%   144  206s
  8108  4135 infeasible   33        71.80154   59.03323  17.8%   144  211s
  8401  4325   62.45710   31  123   71.80154   59.07004  17.7%   144  216s
  8809  4516   59.68962   43  196   71.80154   59.19575  17.6%   144  222s
  9151  4741   63.80180   44  176   71.80154   59.27177  17.5%   143  228s
  9606  4940 infeasible   46        71.80154   59.38366  17.3%   142  233s
 10068  5139   65.21709   40  153   71.80154   59.54127  17.1%   141  239s
*10322  4599              45      69.8329295   59.54228  14.7%   141  239s
 10508  4777   64.96376   48  178   69.83293   59.61548  14.6%   141  246s
 10992  4896   63.00045   46  192   69.83293   59.71960  14.5%   140  252s
 11519  5112   63.92252   25  128   69.83293   59.85017  14.3%   140  259s
 12046  5330   68.90349   42  183   69.83293   59.98165  14.1%   138  265s
 12502  5590   65.85673   39  156   69.83293   60.02640  14.0%   138  272s
 13068  5765   64.79092  

<gurobi.Model MIP instance Quadratic Assignment: 7640 constrs, 8000 vars, Parameter changes: TimeLimit=600.0>

In [24]:
quadratic_assignment_LR('TSP_instance_n_20_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 8000 columns and 23600 nonzeros
Model fingerprint: 0x77264650
Variable types: 7600 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 195.6513711
Presolve time: 0.10s
Presolved: 7640 rows, 8000 columns, 31200 nonzeros
Variable types: 7600 continuous, 400 integer (400 binary)

Root relaxation: objective 0.000000e+00, 275 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   57  195.65137    0.00000   100%     -    0s
H    0     0                     149.9792983    0.00000   100% 

 15872 11851   28.24867   49   94   77.83385   28.13558  63.9%  47.2  356s
 15882 11859   34.24685   50   84   77.83385   28.26403  63.7%  47.3  360s
 15914 11884   28.45038   57   96   77.83385   28.45038  63.4%  47.7  365s
 15961 11922   28.78983   63  105   77.83385   28.78983  63.0%  48.3  370s
 16062 11995   58.05930   81   78   77.83385   30.44788  60.9%  49.3  375s
 16218 12084   70.39447  113   45   77.83385   30.44788  60.9%  50.4  380s
 16449 12193   32.23510   71   93   77.83385   30.50357  60.8%  51.7  385s
 16860 12403   40.24782   70   88   77.83385   30.82081  60.4%  52.9  391s
 17241 12603   38.93050   62   80   77.83385   30.91871  60.3%  53.6  395s
 17582 12780   60.71183  104   53   77.83385   31.02027  60.1%  54.4  400s
 18015 12893   39.78688   66   80   77.83385   32.02611  58.9%  55.2  406s
 18170 13014   53.11359  104   72   77.83385   32.02611  58.9%  56.3  411s
 18472 13159   34.53298   62   82   77.83385   32.28142  58.5%  57.4  416s
 18739 13319     cutoff  

<gurobi.Model MIP instance Quadratic Assignment: 7640 constrs, 8000 vars, Parameter changes: TimeLimit=600.0>

In [25]:
quadratic_assignment('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 8000 columns and 23600 nonzeros
Model fingerprint: 0x718ba896
Variable types: 0 continuous, 8000 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 218.6551184
Presolve time: 0.05s
Presolved: 7640 rows, 8000 columns, 23600 nonzeros
Variable types: 0 continuous, 8000 integer (8000 binary)

Root relaxation: objective 0.000000e+00, 290 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   60  218.65512    0.00000   100%     -    0s
H    0     0                     165.1972964    0.00000   100%    

 10475  4942   73.00579   36  483   78.55552   67.97159  13.5%   137  265s
 10478  4944   71.62290   26  501   78.55552   67.97159  13.5%   137  270s
 10481  4946   76.96938   44  492   78.55552   67.97159  13.5%   137  275s
 10483  4947   71.88316   25  489   78.55552   67.98104  13.5%   137  304s
 10484  4951   67.98104   29  427   78.55552   67.98104  13.5%   138  310s
 10486  4954   67.98104   30  439   78.55552   67.98104  13.5%   138  318s
 10490  4957   67.98104   31  444   78.55552   67.98104  13.5%   138  322s
 10494  4960   68.73325   31  244   78.55552   67.98104  13.5%   138  327s
 10498  4962   67.98104   32  443   78.55552   67.98104  13.5%   138  332s
 10502  4965   67.98104   32  227   78.55552   67.98104  13.5%   139  335s
 10511  4972   67.98104   33  432   78.55552   67.98104  13.5%   139  341s
 10523  4982   67.98104   35  362   78.55552   67.98104  13.5%   140  346s
 10538  4992   67.98104   37  239   78.55552   67.98104  13.5%   140  350s
 10560  5008   67.98104  

<gurobi.Model MIP instance Quadratic Assignment: 7640 constrs, 8000 vars, Parameter changes: TimeLimit=600.0>

In [26]:
quadratic_assignment_LR('TSP_instance_n_20_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7640 rows, 8000 columns and 23600 nonzeros
Model fingerprint: 0xc3d66313
Variable types: 7600 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 218.6551184
Presolve time: 0.07s
Presolved: 7640 rows, 8000 columns, 31200 nonzeros
Variable types: 7600 continuous, 400 integer (400 binary)

Root relaxation: objective 0.000000e+00, 275 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   57  218.65512    0.00000   100%     -    0s
H    0     0                     208.0408299    0.00000   100% 

 15759 12186   37.37231   98  122  103.37806   37.37231  63.8%  41.8  350s
 15761 12190   37.43468   45  111  103.37806   37.43468  63.8%  43.8  356s
 15763 12192   37.52200   46  153  103.37806   37.52200  63.7%  43.9  362s
 15765 12193   40.37836   46  110  103.37806   37.55732  63.7%  44.0  367s
 15767 12194   37.61711   47  122  103.37806   37.61711  63.6%  44.0  372s
 15769 12196   45.99959   47  111  103.37806   37.66539  63.6%  44.1  377s
 15773 12199   40.94492   48   91  103.37806   37.80458  63.4%  44.2  381s
 15780 12204   38.57021   50   96  103.37806   37.95967  63.3%  44.4  385s
 15797 12219   38.85961   53   96  103.37806   38.85961  62.4%  44.7  390s
 15837 12247   40.68403   61  103  103.37806   39.84124  61.5%  45.4  395s
 15872 12272   46.60910   67   81  103.37806   39.84124  61.5%  46.0  400s
 15940 12325   44.35210   80   74  103.37806   39.84124  61.5%  46.9  405s
 16091 12414   53.96316  101   67  103.37806   39.84124  61.5%  47.9  411s
 16216 12497   78.06146  

<gurobi.Model MIP instance Quadratic Assignment: 7640 constrs, 8000 vars, Parameter changes: TimeLimit=600.0>

The following are the Quadratic Assignment formulation integer and Linear Relaxation solutions for two instances of size 25.

In [27]:
quadratic_assignment('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15625 columns and 46250 nonzeros
Model fingerprint: 0xb15b2a43
Variable types: 0 continuous, 15625 integer (625 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 206.7392421
Presolve time: 0.11s
Presolved: 15050 rows, 15625 columns, 46250 nonzeros
Variable types: 0 continuous, 15625 integer (15625 binary)

Root relaxation: objective 0.000000e+00, 462 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   76  206.73924    0.00000   100%     -    0s
     0     0    0.00000    0   79  206.73924    0.00000   1

  1039   815   70.66890   74  243   93.84687   67.67650  27.9%   296  278s
  1052   829   69.20315   77  365   93.84687   67.67650  27.9%   302  289s
  1074   844   69.43804   83  353   93.84687   67.67650  27.9%   310  294s
  1097   860   69.57748   88  339   93.84687   67.67650  27.9%   317  300s
  1121   881   69.58484   93  342   93.84687   67.67650  27.9%   325  305s
  1150   901   80.13763   98  356   93.84687   67.67650  27.9%   332  313s
  1179   937   69.97404  103  344   93.84687   67.67650  27.9%   337  320s
  1229   951   72.10270  111  344   93.84687   67.67650  27.9%   341  327s
  1280   953   70.47896  118  336   93.84687   67.67650  27.9%   343  333s
  1299   968   72.04706  124  248   93.84687   67.67650  27.9%   350  343s
  1322   983   76.25830  132  231   93.84687   67.67650  27.9%   359  351s
  1345  1004   78.47540  140  218   93.84687   67.67650  27.9%   367  356s
  1375  1032   81.93045  150  219   93.84687   67.67650  27.9%   373  362s
  1446  1033 infeasible  

<gurobi.Model MIP instance Quadratic Assignment: 15050 constrs, 15625 vars, Parameter changes: TimeLimit=600.0>

In [28]:
quadratic_assignment_LR('TSP_instance_n_25_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15625 columns and 46250 nonzeros
Model fingerprint: 0x83ffffd8
Variable types: 15000 continuous, 625 integer (625 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 206.7392421
Presolve time: 0.45s
Presolved: 15050 rows, 15625 columns, 61236 nonzeros
Variable types: 15000 continuous, 625 integer (625 binary)

Root relaxation: objective 0.000000e+00, 464 iterations, 0.18 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   79  206.73924    0.00000   100%     -    2s
H    0     0                     205.4876337    0.00000  

 37788 32253   33.37641   69  101  106.66339    2.94920  97.2%  70.2  400s
 38677 33043   12.01689  101   99  106.66339    2.97769  97.2%  69.8  406s
 39351 33616   22.97357  107  100  106.66339    2.97769  97.2%  69.6  410s
 40109 34338    8.72268   74  118  106.66339    2.97769  97.2%  69.7  415s
 40866 34916   50.25537  157   95  106.66339    2.97769  97.2%  69.8  421s
H41211 34892                     105.5680557    2.97769  97.2%  69.6  422s
 41404 35210   43.07407  133   65  105.56806    2.97769  97.2%  69.8  425s
 42423 36107   89.03868  176   42  105.56806    3.46344  96.7%  69.3  430s
 43379 36930   10.85276  100  117  105.56806    3.46344  96.7%  69.0  436s
 43987 37479   52.78725  134   76  105.56806    3.46344  96.7%  69.0  440s
 44947 38285   47.41028   89   79  105.56806    3.47032  96.7%  68.6  446s
 45513 38783   23.78866   71  100  105.56806    3.49771  96.7%  68.7  450s
H46303 39397                     105.0741431    3.59329  96.6%  68.6  455s
 46870 39813    8.88620  

<gurobi.Model MIP instance Quadratic Assignment: 15050 constrs, 15625 vars, Parameter changes: TimeLimit=600.0>

In [31]:
# RUN HERE
quadratic_assignment('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15625 columns and 46250 nonzeros
Model fingerprint: 0x52413210
Variable types: 0 continuous, 15625 integer (625 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 251.0090057
Presolve time: 0.14s
Presolved: 15050 rows, 15625 columns, 46250 nonzeros
Variable types: 0 continuous, 15625 integer (15625 binary)

Root relaxation: objective 0.000000e+00, 349 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   66  251.00901    0.00000   100%     -    0s
H    0     0                     192.0217892    0.00000   1

H  884   746                     117.9399063   68.66790  41.8%   198  153s
   895   761   68.67301   40  381  117.93991   68.67301  41.8%   201  155s
H  932   745                     105.0899223   68.67742  34.6%   207  157s
   954   774   68.69379   46  364  105.08992   68.69379  34.6%   210  160s
H  983   740                     101.2242586   68.71786  32.1%   214  161s
  1032   798   68.87350   53  379  101.22426   68.78051  32.1%   216  165s
  1124   827   68.92935   61  390  101.22426   68.78051  32.1%   220  171s
  1171   865   69.16164   67  391  101.22426   68.78051  32.1%   229  175s
  1245   920   69.23394   74  348  101.22426   68.78051  32.1%   235  180s
  1355   941   69.36792   82  407  101.22426   68.78051  32.1%   236  186s
  1404   984   72.13550   86  332  101.22426   68.78051  32.1%   237  190s
  1467  1014   72.66934   96  343  101.22426   68.78051  32.1%   244  197s
  1512  1055   72.84570  103  330  101.22426   68.78051  32.1%   248  201s
  1571  1115   74.11407  

<gurobi.Model MIP instance Quadratic Assignment: 15050 constrs, 15625 vars, Parameter changes: TimeLimit=600.0>

In [32]:
quadratic_assignment_LR('TSP_instance_n_25_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 15050 rows, 15625 columns and 46250 nonzeros
Model fingerprint: 0x72a81973
Variable types: 15000 continuous, 625 integer (625 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 251.0090057
Presolve time: 0.16s
Presolved: 15050 rows, 15625 columns, 61236 nonzeros
Variable types: 15000 continuous, 625 integer (625 binary)

Root relaxation: objective 0.000000e+00, 426 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   64  251.00901    0.00000   100%     -    0s
     0     0    0.00000    0   50  251.00901    0.00000  

<gurobi.Model MIP instance Quadratic Assignment: 15050 constrs, 15625 vars, Parameter changes: TimeLimit=600.0>

The following are the Quadratic Assignment formulation integer and Linear Relaxation solutions for two instances of size 30.

In [33]:
quadratic_assignment('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 27000 columns and 80100 nonzeros
Model fingerprint: 0xcb908b1a
Variable types: 0 continuous, 27000 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 242.0323393
Presolve time: 0.22s
Presolved: 26160 rows, 27000 columns, 80100 nonzeros
Variable types: 0 continuous, 27000 integer (27000 binary)

Root relaxation: objective 0.000000e+00, 586 iterations, 0.19 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  104  242.03234    0.00000   100%     -    1s
     0     0    0.00000    0   71  242.03234    0.00000   1

   904   845   86.74331   69  146  130.35514   74.02207  43.2%   195  235s
   908   848   81.27604  113  231  130.35514   74.02207  43.2%   194  240s
H  908   805                     127.9966169   74.02207  42.2%   194  245s
   911   807   82.26672  123  483  127.99662   74.02207  42.2%   194  250s
   913   808  125.50520  311  488  127.99662   74.02207  42.2%   193  256s
   915   809  104.27289  230  561  127.99662   74.47040  41.8%   193  261s
   917   811   79.40687   38  556  127.99662   74.84135  41.5%   192  265s
   920   813  119.47475  339  661  127.99662   75.03316  41.4%   192  272s
   922   814  126.32578  353  701  127.99662   75.22394  41.2%   191  276s
   924   815  111.47700  317  785  127.99662   75.38034  41.1%   191  280s
   925   816  113.49529  330  785  127.99662   75.43427  41.1%   191  293s
   926   817   83.00549  103  781  127.99662   75.43478  41.1%   191  296s
   928   818  106.62319  272  744  127.99662   75.49406  41.0%   190  300s
   930   819   88.40173  

<gurobi.Model MIP instance Quadratic Assignment: 26160 constrs, 27000 vars, Parameter changes: TimeLimit=600.0>

In [34]:
quadratic_assignment_LR('TSP_instance_n_30_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 27000 columns and 80100 nonzeros
Model fingerprint: 0x7775ea40
Variable types: 26100 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 242.0323393
Presolve time: 0.29s
Presolved: 26160 rows, 27000 columns, 104248 nonzeros
Variable types: 26100 continuous, 900 integer (900 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 576 iterations, 0.31 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

 13607 11847   26.56788  375   76  129.30929    0.00000   100%   118  408s
 13942 12208   13.15136  151   92  129.30929    0.00000   100%   117  413s
 14323 12491  128.70285  365   54  129.30929    0.00000   100%   116  417s
 14669 12685   24.88377  208  104  129.30929    0.00000   100%   115  494s
H14894 12953                     128.8805855    0.00000   100%   115  498s
 15211 13043 infeasible  336       128.88059    0.00000   100%   114  502s
 15323 13253   13.02794  194   99  128.88059    0.00000   100%   115  506s
 15562 13492    5.62724  214  111  128.88059    0.00000   100%   115  510s
 16089 13935   16.78214  251  105  128.88059    0.00000   100%   114  518s
 16328 14228   62.37235  289   71  128.88059    0.00000   100%   113  522s
 16637 14474   79.83518  345   68  128.88059    0.00000   100%   113  526s
H16945 14699                     128.5026713    0.00000   100%   112  530s
 17592 15464   79.58741  302   60  128.50267    0.00000   100%   111  538s
 18045 15764   47.99053  

<gurobi.Model MIP instance Quadratic Assignment: 26160 constrs, 27000 vars, Parameter changes: TimeLimit=600.0>

In [35]:
quadratic_assignment('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 27000 columns and 80100 nonzeros
Model fingerprint: 0xaa23a50a
Variable types: 0 continuous, 27000 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 349.4293230
Presolve time: 0.21s
Presolved: 26160 rows, 27000 columns, 80100 nonzeros
Variable types: 0 continuous, 27000 integer (27000 binary)

Root relaxation: objective 0.000000e+00, 586 iterations, 0.17 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  104  349.42932    0.00000   100%     -    1s
H    0     0                     316.5958374    0.00000   1

H  979   751                     149.9771737   64.87237  56.7%   182  231s
   988   767   65.51244   59  452  149.97717   64.87237  56.7%   184  237s
  1000   773   65.52074   62  447  149.97717   64.87237  56.7%   187  240s
  1010   781   66.07457   64  517  149.97717   64.87237  56.7%   189  245s
  1022   781   66.47867   67  432  149.97717   64.87237  56.7%   192  256s
  1026   793   66.52185   68  429  149.97717   64.87237  56.7%   193  260s
  1054   818   66.93874   74  389  149.97717   64.87237  56.7%   198  268s
  1073   829   68.11939   78  274  149.97717   64.87237  56.7%   202  272s
  1091   841   67.10424   82  337  149.97717   64.87237  56.7%   203  277s
  1110   859   67.34525   87  308  149.97717   64.87237  56.7%   205  282s
  1135   876   67.53666   93  341  149.97717   64.87237  56.7%   208  287s
  1160   892   67.83372   99  303  149.97717   64.87237  56.7%   211  292s
  1186   913   67.99185  105  329  149.97717   64.87237  56.7%   214  297s
  1217   928   70.67988  

<gurobi.Model MIP instance Quadratic Assignment: 26160 constrs, 27000 vars, Parameter changes: TimeLimit=600.0>

In [36]:
quadratic_assignment_LR('TSP_instance_n_30_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 26160 rows, 27000 columns and 80100 nonzeros
Model fingerprint: 0x2088bdfb
Variable types: 26100 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 349.4293230
Presolve time: 0.34s
Presolved: 26160 rows, 27000 columns, 104248 nonzeros
Variable types: 26100 continuous, 900 integer (900 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 535 iterations, 0.32 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

  9420  7604    4.86983   69   85  167.05036    0.00000   100%   131  342s
  9496  7683    8.61172   84   67  167.05036    0.00000   100%   133  346s
  9575  7756    0.00000  102   74  167.05036    0.00000   100%   134  350s
  9721  7902    0.00000  133   60  167.05036    0.00000   100%   138  358s
  9794  7979   15.83421  148  157  167.05036    0.00000   100%   140  362s
  9871  8011    0.00000  165   73  167.05036    0.00000   100%   141  367s
  9903  8110    6.96966  170  111  167.05036    0.00000   100%   142  371s
 10002  8190    6.00183  185  135  167.05036    0.00000   100%   144  375s
 10082  8315    5.23984  197  112  167.05036    0.00000   100%   145  380s
 10324  8574    0.00000  225   72  167.05036    0.00000   100%   146  389s
 10501  8774    5.51913  242  108  167.05036    0.00000   100%   146  393s
 10717  8914   14.53628  258   80  167.05036    0.00000   100%   145  401s
 10857  9077   14.76723  275  139  167.05036    0.00000   100%   146  405s
 11020  9283    5.52081  

<gurobi.Model MIP instance Quadratic Assignment: 26160 constrs, 27000 vars, Parameter changes: TimeLimit=600.0>

The following are the Quadratic Assignment formulation integer and Linear Relaxation solutions for two instances of size 50.

In [37]:
quadratic_assignment('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 125000 columns and 372500 nonzeros
Model fingerprint: 0xe3286525
Variable types: 0 continuous, 125000 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 516.1287467
Presolve time: 1.28s
Presolved: 122600 rows, 125000 columns, 372500 nonzeros
Variable types: 0 continuous, 125000 integer (125000 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 1101 iterations, 1.72 seconds
Total elapsed time = 7.06s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

<gurobi.Model MIP instance Quadratic Assignment: 122600 constrs, 125000 vars, Parameter changes: TimeLimit=600.0>

In [42]:
quadratic_assignment_LR('TSP_instance_n_50_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 125000 columns and 372500 nonzeros
Model fingerprint: 0x00faef9a
Variable types: 122500 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 516.1287467
Presolve time: 1.74s
Presolved: 122600 rows, 125000 columns, 440814 nonzeros
Variable types: 122500 continuous, 2500 integer (2500 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 1375 iterations, 2.16 seconds
Total elapsed time = 7.68s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

<gurobi.Model MIP instance Quadratic Assignment: 122600 constrs, 125000 vars, Parameter changes: TimeLimit=600.0>

In [43]:
quadratic_assignment('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 125000 columns and 372500 nonzeros
Model fingerprint: 0xc3b62740
Variable types: 0 continuous, 125000 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 520.8259300
Presolve time: 1.29s
Presolved: 122600 rows, 125000 columns, 372500 nonzeros
Variable types: 0 continuous, 125000 integer (125000 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.02s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 1181 iterations, 1.71 seconds
Total elapsed time = 5.41s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

<gurobi.Model MIP instance Quadratic Assignment: 122600 constrs, 125000 vars, Parameter changes: TimeLimit=600.0>

In [44]:
quadratic_assignment_LR('TSP_instance_n_50_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 122600 rows, 125000 columns and 372500 nonzeros
Model fingerprint: 0x01fcfd77
Variable types: 122500 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 520.8259300
Presolve time: 1.67s
Presolved: 122600 rows, 125000 columns, 440814 nonzeros
Variable types: 122500 continuous, 2500 integer (2500 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 1448 iterations, 2.33 seconds
Total elapsed time = 7.34s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

<gurobi.Model MIP instance Quadratic Assignment: 122600 constrs, 125000 vars, Parameter changes: TimeLimit=600.0>

The following are the Quadratic Assignment formulation integer and Linear Relaxation solutions for two instances of size 70.

In [45]:
quadratic_assignment('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 343000 columns and 1024100 nonzeros
Model fingerprint: 0x2c43b9de
Variable types: 0 continuous, 343000 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 662.8343047
Presolve time: 4.30s
Presolved: 338240 rows, 343000 columns, 1024100 nonzeros
Variable types: 0 continuous, 343000 integer (343000 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
  338971    3.8698404e+02   0.000000e+00   4.526271e+05     10s
Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 2379 ite

<gurobi.Model MIP instance Quadratic Assignment: 338240 constrs, 343000 vars, Parameter changes: TimeLimit=600.0>

In [46]:
quadratic_assignment_LR('TSP_instance_n_70_s_1.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 343000 columns and 1024100 nonzeros
Model fingerprint: 0x0dcc9b48
Variable types: 338100 continuous, 4900 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 662.8343047
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve time: 6.02s
Presolved: 338240 rows, 343000 columns, 1242796 nonzeros
Variable types: 338100 continuous, 4900 integer (4900 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
  338709    7.5032245e+02   0.000000e+00   2.574658e+05     10s
Concurrent spin time: 0.00s

Solved w

<gurobi.Model MIP instance Quadratic Assignment: 338240 constrs, 343000 vars, Parameter changes: TimeLimit=600.0>

In [47]:
quadratic_assignment('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 343000 columns and 1024100 nonzeros
Model fingerprint: 0x0bdfbac5
Variable types: 0 continuous, 343000 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 715.5626965
Presolve time: 4.25s
Presolved: 338240 rows, 343000 columns, 1024100 nonzeros
Variable types: 0 continuous, 343000 integer (343000 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
  338871    3.8590891e+02   0.000000e+00   5.129859e+05     10s
Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 0.000000e+00, 1247 ite

<gurobi.Model MIP instance Quadratic Assignment: 338240 constrs, 343000 vars, Parameter changes: TimeLimit=600.0>

In [48]:
quadratic_assignment_LR('TSP_instance_n_70_s_2.dat')

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 338240 rows, 343000 columns and 1024100 nonzeros
Model fingerprint: 0x2850a953
Variable types: 338100 continuous, 4900 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 715.5626965
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve time: 6.29s
Presolved: 338240 rows, 343000 columns, 1242796 nonzeros
Variable types: 338100 continuous, 4900 integer (4900 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
  338409    7.4697507e+02   0.000000e+00   1.962202e+06     10s
Concurrent spin time: 0.00s

Solved w

<gurobi.Model MIP instance Quadratic Assignment: 338240 constrs, 343000 vars, Parameter changes: TimeLimit=600.0>